# Project Part b)

In [1]:
from fenics import *
from dolfin import *
from mshr import *

import matplotlib
%matplotlib qt
#%matplotlib widget
#%matplotlib inline
#matplotlib.rcParams['figure.figsize'] = 15, 10

import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(threshold=np.inf, linewidth=300)
from scipy import sparse


from mpl_toolkits.mplot3d import Axes3D

In [8]:
def StiffnessAssembler2D(p, t, a):
    '''
    input p is mesh.coordinates(), dimension (np, 2)
    input t is mesh.cells(), dimension (nt, 3)
    input a is the coefficient function a(x,y)
    output A is the stiffness matrix
    '''
    n_p = np.shape(p)[0]       # number of nodes
    n_t = np.shape(t)[0]       # number of elements
    A = sparse.lil_matrix((n_p, n_p))   # allocate stiffnessmatrix
    for K in np.arange(n_t):
        loc2glb = t[K] 
        x = p[loc2glb][:,0]   # node x-coordinates
        y = p[loc2glb][:,1]   #      y
        area, b, c = HatGradients(x, y)
        xc = np.mean(x)
        yc = np.mean(y)       # element centroid
        abar = a(xc, yc)      # value of a(x,y) at centroid
        AK = abar*(np.outer(b,b) + np.outer(c,c))*area # element stiffness matrix
        A[np.ix_(loc2glb, loc2glb)] += AK
    
    return A


def LoadAssembler2D(p, t, f):
    n_p = np.shape(p)[0]       # number of nodes
    n_t = np.shape(t)[0]       # number of boundary edges
    b = np.zeros(n_p)
    for K in np.arange(n_t):
        loc2glb = t[K]
        x = p[loc2glb][:,0]   # node x-coordinates
        y = p[loc2glb][:,1]   #      y
        area = polyArea(x, y);               # area of the triangle
        bK = np.array([f(x[0], y[0]),
                       f(x[1], y[1]),
                       f(x[2], y[2])])/3*area # element load vector
        b[np.ix_(loc2glb)] += bK
    
    return b   


def assemblor(p, e, t, f, gamma, g):
    n_p = np.shape(p)[0]       # number of nodes
    n_t = np.shape(t)[0]       # number of elements
    n_e = np.shape(e)[0]       # number of boundary edges
    A = sparse.lil_matrix((n_p, n_p))   # allocate stiffnessmatrix
    R = sparse.lil_matrix((n_p, n_p))   # allocate Robin mass matrix
    b = np.zeros(n_p)          # load vector
    r = np.zeros(n_p)          # Robin load vector
    
    for K in np.arange(n_t):
        loc2glb = t[K]
        x = p[loc2glb][:,0]   # node x-coordinates
        y = p[loc2glb][:,1]   #      y
        area, bi, ci = HatGradients(x, y)
        AK = (np.outer(bi,bi) + np.outer(ci,ci))*area
        bK = np.array([f(x[0], y[0]),
                       f(x[1], y[1]),
                       f(x[2], y[2])])/3*area
        A[np.ix_(loc2glb, loc2glb)] += AK
        b[np.ix_(loc2glb)] += bK
        
    
    for E in np.arange(n_e):
        loc2glb = e[E]        # local-to-global map
        x = p[loc2glb][:,0]   # node x-coordinates
        y = p[loc2glb][:,1]   #      y
        z1 = gamma(x, y)
        z2 = g(x, y)
        len = np.sqrt((x[0]-x[1])**2 + (y[0]-y[1])**2)  # edge length
        RE = np.array([[2, 1,],
                       [1, 2]])/6*len*z1    # edge boundary matrix
        
        rE = z1*len*z2/2
        R[np.ix_(loc2glb, loc2glb)] += RE    # Add element masses to R
        r[np.ix_(loc2glb)] += rE
    
    return A, R, b, r

    
def polyArea(x, y):
    v1 = np.array([x[1]-x[0], y[1]-y[0]])
    v2 = np.array([x[2]-x[0], y[2]-y[0]])
    area = 1/2 * np.linalg.norm(np.cross(v1, v2))
    return area


def HatGradients(x, y):
    ''' Returns the area, b, and c
        Grad(ph_i) = transpose([b_i, c_i])
    '''
    area = polyArea(x, y);
    b = np.array([y[1]-y[2], y[2]-y[0], y[0]-y[1]])/2/area
    c = np.array([x[2]-x[1], x[0]-x[2], x[1]-x[0]])/2/area
    return area, b, c

In [2]:
domain = Circle(Point(0.,0.),1.0,60)
mesh = generate_mesh(domain, 5, "cgal")
plot(mesh)
plt.show()

p = mesh.coordinates()

# Obtain the e matrix
e_points = BoundaryMesh(mesh, "exterior", True)
e_indices = e_points.entity_map(0)
e = e_indices.array()
e2 = np.append(e[1:], e[0])
e = np.dstack((e, e2)).squeeze()
t = mesh.cells()

In [9]:
I = np.eye(len(p))
#A, R, b, r = 

# Problem B.1

In [ ]:
def f(x, y):
    return 8 * np.pi**2 * np.sin(2*np.pi*x) * np.sin(2*np.pi*y)

def gamma(x, y):            # <--------------
    return 1e6 if x.any()==0 else 0

def g(x, y):                # <----------------
    z = 0
    z = np.sin(2*np.pi*y)
    return z